In [13]:
from openai import OpenAI
from dateutil.relativedelta import relativedelta
import os
import json
from pydantic import BaseModel,Field
from langchain.llms import OpenAI
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI, init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
from tqdm import tqdm
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
import langsmith as ls
from langsmith import traceable, trace
from langsmith import Client, traceable, evaluate
from preprocess_data import prepare_qa_input_with_answer_filter,standardize_item

In [14]:
file_path = '../data/GSM8K/test.jsonl'

with open(file_path, "r", encoding="utf-8") as f:
    gsm8k = [json.loads(line) for line in f]

In [15]:
folder_path = "../dataset_langsmith/"
filename="tatqa_filtered.json"
file_path = os.path.join(folder_path, filename)
with open(file_path, "r", encoding="utf-8") as f:
    raw_data_tatqa = json.load(f)
tatqa = prepare_qa_input_with_answer_filter(raw_data_tatqa)

In [16]:
folder_path = "../dataset_langsmith/"
filename="tabmwp_filtered.json"
file_path = os.path.join(folder_path, filename)
with open(file_path, "r", encoding="utf-8") as f:
    tabmwp = json.load(f)

In [17]:
name= "gsm8k" # tatqa, tabmwp
length_test= 300 # số lượng mẫu muốn test
if name == "gsm8k":
   DATA=gsm8k 
   name_model="GSM8K"
elif name == "tatqa":
   DATA=tatqa
   name_model="TATQA"
else:
   DATA=tabmwp
   name_model="TABMWP"

In [18]:
load_dotenv()
model=init_chat_model('gpt-4.1-mini',model_provider='openai',temperature=0.2)

In [19]:
class MathReasoning(BaseModel):
    explanation: str
    final_answer: str

In [20]:
model_with_tools = model.with_structured_output(MathReasoning)

In [21]:
def extract_ground_truth(answer, dataset_type):
    if dataset_type == "gsm8k":
        match = re.search(r"####\s*([\d,./]+)", answer)
        if match:
            return match.group(1).replace(",", "").strip()
        return answer.strip()
    elif dataset_type == "tatqa":
        # Có thể là list hoặc chuỗi, lấy phần đầu tiên nếu là list
        if isinstance(answer, list):
            ans = str(answer[0]).strip()
        else:
            ans = str(answer).strip()
        # Nếu là dạng [2019] hoặc ["2019"], loại bỏ ngoặc vuông và dấu nháy
        ans = re.sub(r'^[\[\"]*([\d\-\.\/]+)[\]\"]*$', r'\1', ans)
        # Nếu là phân số thì giữ lại dấu "/"
        if '/' in ans:
            ans = re.sub(r"[^-\d/\.]", "", ans)
        else:
            ans = re.sub(r"[^-\d\.]", "", ans)
        return ans
    else:
        return str(answer).strip()
    
def compare_answers(predicted: str, actual: str, eps: float = 1e-3) -> bool:
    try:
        pred = round(float(predicted.strip()))
        act = round(float(actual.strip()))
        return abs(pred - act) < eps
    except ValueError:
        return predicted.strip().lower() == actual.strip().lower()

In [22]:
def process_item(item,dataset_type):
    question = item["question"]
    context = item.get("context", "")
    true_answer = extract_ground_truth(item["answer"],dataset_type)
    try:
        # Nếu có context, nối vào trước question
        if context.strip():
            user_content = f"# Context:\n{context}\n\n# Question: {question}"
        else:
            user_content = question

        messages = [
            SystemMessage(content="""
            You are a math expert.
            For every question and context, you **must** respond using the `MathReasoning` tool.
            - Do not respond with plain text or natural language.
            - Include a `final_answer` as a single number, no units or symbols.
            - If you cannot solve it, return a final_answer of "unknown".
            - When dealing with money, do not round to thousands unless explicitly stated.
            """),
            HumanMessage(content=user_content)
        ]
        ai_msg = model_with_tools.invoke(messages)
        predicted_answer = ai_msg.final_answer

        return {
            "question": question,
            "context": context,
            "true_answer": true_answer,
            "predicted_answer": predicted_answer,
            "correct": compare_answers(predicted_answer, true_answer)
        }
    except Exception as e:
        return {"error": str(e), "question": question}

dataset = []
for item in DATA:
    dataset.extend(standardize_item(item, f"{name}"))

results = []
correct = 0
total = len(dataset[:length_test])
with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(process_item, item, f"{name}") for item in dataset[:length_test]]
    for future in tqdm(as_completed(futures), total=total):
        result = future.result()
        if "error" not in result:
            results.append(result)
            if result["correct"]:
                correct += 1
        else:
            print(f"Error on question: {result['question'][:60]}... => {result['error']}")
accuracy = correct / total * 100
print(f"Accuracy: {accuracy:.2f}% ({correct}/{total})")

 40%|████      | 120/300 [00:57<01:54,  1.57it/s]Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=ef69a8aa-50f5-427e-b550-c0bcb481c422,id=40093593-bf11-496a-ab3e-22414a3dc162; trace=ef69a8aa-50f5-427e-b550-c0bcb481c422,id=3f0cd403-c0e6-413d-91de-a9267ed69b9c; trace=ef69a8aa-50f5-427e-b550-c0bcb481c422,id=3f0cd403-c0e6-413d-91de-a9267ed69b9c; trace=ef69a8aa-50f5-427e-b550-c0bcb481c422,id=ef69a8aa-50f5-427e-b550-c0bcb481c422; trace=1dc23fc3-5b71-43a9-95ff-39d5abd0c32f,id=1dc23fc3-5b71-43a9-95ff-39d5abd0c32f; trace=1dc23fc3-5b71-43a9-95ff-39d5abd0c32f,id=e4cb3c7a-5b1b-4c2e-b6e2-77d27b006653
Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimit

Accuracy: 97.00% (291/300)


In [23]:
output_path = "Zero_shot_results.json"
os.makedirs(os.path.dirname(output_path) or ".", exist_ok=True)
def custom_encoder(obj):
    if hasattr(obj, "model_dump"):
        return obj.model_dump()
    elif hasattr(obj, "dict"):
        return obj.dict()
    else:
        raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

wrong_answers = [r for r in results if not r.get("correct", False)]

try:
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(wrong_answers, f, ensure_ascii=False, indent=2, default=custom_encoder)
    print(f"Đã lưu {len(wrong_answers)} kết quả sai vào {output_path}")
except TypeError as e:
    print(f"Lỗi khi ghi file JSON: {e}")


Đã lưu 9 kết quả sai vào Zero_shot_results.json


In [24]:
import math
@traceable(run_type="chain")
def target_function(inputs: dict):
    question = inputs["question"]
    context = inputs.get("context", "")
    # Nếu có context, nối vào trước question
    if context.strip():
        user_content = f"# Context:\n{context}\n\n# Question: {question}"
    else:
        user_content = question

    messages = [
        SystemMessage(content="""
        You are a math expert.
        For every question, you **must** respond using the `MathReasoning` tool.
        - Do not respond with plain text or natural language.
        - Include a `final_answer` as a single number, no units or symbols.
        - If you cannot solve it, return a final_answer of "unknown".
        - When dealing with money, do not round to thousands unless explicitly stated.
        """),
        HumanMessage(content=user_content)
    ]
    ai_msg = model_with_tools.invoke(messages)
    predicted_answer = ai_msg.final_answer
    # Nếu muốn log reasoning steps
    return {
        "final_answer": predicted_answer,
        "steps": getattr(ai_msg, "steps", None)
    }

@traceable(run_type="tool")
def compare_result(inputs: dict, reference_outputs: dict, outputs: dict):
    reference_response = extract_ground_truth(reference_outputs["answer"],f"{name}")
    run_response = outputs.get("final_answer")
    reference_response = str(reference_response).strip()
    run_response = str(run_response).strip()
    try:
        score = math.isclose(float(reference_response), float(run_response), rel_tol=1e-3)
    except Exception:
        score = (reference_response == run_response)
    return {"key": "is_correct", "score": int(score)}

client = Client()
evaluate(
    target_function,
    data=client.list_examples(dataset_name=f"{name_model}", splits=["base"]),
    evaluators=[compare_result],
    experiment_prefix=f"{name_model} - Test Dataset"
)


Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=5396c53c-743a-421e-bb77-a4e2fcc9491d,id=04136668-2b0e-4140-901a-439914448754; trace=5396c53c-743a-421e-bb77-a4e2fcc9491d,id=7b7f8d24-cdee-4259-a542-89083869550f; trace=5396c53c-743a-421e-bb77-a4e2fcc9491d,id=7b7f8d24-cdee-4259-a542-89083869550f; trace=5396c53c-743a-421e-bb77-a4e2fcc9491d,id=5396c53c-743a-421e-bb77-a4e2fcc9491d


View the evaluation results for experiment: 'GSM8K - Test Dataset-ff3b6fc8' at:
https://smith.langchain.com/o/e7b5e917-6c40-46ad-a54b-83be84870fd4/datasets/a0f9251e-0ade-4046-a15d-3eb36e10746f/compare?selectedSessions=01f1130b-3a07-49d5-9e1a-add13ac3543d




0it [00:00, ?it/s]

Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=8701ccef-910d-4162-bb5e-708c11552ab3,id=8701ccef-910d-4162-bb5e-708c11552ab3; trace=8701ccef-910d-4162-bb5e-708c11552ab3,id=255ed645-2609-4740-966e-02c1a0fd5969; trace=8701ccef-910d-4162-bb5e-708c11552ab3,id=dc9e24a4-9c4b-4cd5-98b6-30472146cc40
Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=870

,inputs.question,outputs.final_answer,outputs.steps,error,reference.answer,feedback.is_correct,execution_time,example_id,id
0,Becky bought 20 apples for 45 cents each and r...,1,None,None,"Before the discount, Becky would have paid 20 ...",1,2.776933,01525728-4311-4402-b357-ff84108cf744,8701ccef-910d-4162-bb5e-708c11552ab3
1,The bananas at the supermarket cost $0.80 each...,2,None,None,Ten bunches makes 10*3=$<<10*3=30>>30.\nThere ...,1,2.893845,034612d6-f271-4086-8803-c284072c82bf,839bbdad-201d-4069-a839-a7cac65792e7
2,Verna loves to eat fruit. She bought three app...,7,None,None,Three apples cost 3 x $1.50 = $<<3*1.5=4.50>>4...,1,2.663023,03e33507-0abf-476f-bd85-5eaaad4f87f9,7452627d-0add-45b4-af07-4e51c8fbea03
3,Emma can make and upload 72 vlogs per month. B...,18,None,None,Emma was able to make 18 + 21 + 15 = <<18+21+1...,1,1.719172,041a29d6-665d-46f8-af8c-ca74d776f0c9,3990a7b4-6932-492d-892c-53ef62bed6b5
4,"The combined age of Peter, Paul and Jean is 10...",50,None,None,Let x be the age of John. Paul’s age is x + 10...,1,3.986518,06562ae2-9130-4f38-b334-265cc66ef8da,291d8139-925c-4fd3-9a32-c096450a0bdb
...,...,...,...,...,...,...,...,...,...
295,A customer’s loyalty card at a store gives the...,31,None,None,The customer had 80 / 20 = $<<80/20=4>>4 in re...,1,2.185655,fc37b2c2-feda-46cb-80ab-69628352c1e3,318bf768-4abb-49e2-8acd-720e89092033
296,Edgar eats 18 pretzels a day. If his brother e...,63,None,None,His brother eats 9 pretzels a day because 18 /...,1,1.971148,fccfc440-10fb-406b-8804-83951929c7f3,be6fa20c-7ede-420f-b1cf-a7452075c8e2
297,Madeline ate 6 grapes. Her brother wanted to m...,84,None,None,Madeline's brother used 6 x 5 = <<6*5=30>>30 g...,1,2.923875,fd267da6-b5f4-4d1d-9818-a88068193dd6,046d5735-004c-499f-bccf-2e15ebb94094
298,"As Sally walked to school, she was holding the...",34,None,None,"Sally started out with 25 red, 7 green, and 12...",1,3.526764,fd4425c2-c834-4e06-ba24-d31e18111299,d6c3d735-6840-45b6-8348-ce797518c968
